#### Task 1: Simple vector embedding generation

**Objective:**
Generate vector embeddings from text data.

**Task Description:**

- load huggingface embedding model (``model_name="sentence-transformers/all-mpnet-base-v2"``)
- embed simple text queries

How to select the right embedding model: [MTEB - Massive Text Embedding Benchmark](https://huggingface.co/blog/mteb)

**Useful links:**

- [Langchain Chroma](https://python.langchain.com/v0.2/docs/integrations/vectorstores/chroma/)


In [1]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.documents import Document

# ADD HERE YOUR CODE
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

c:\gamemode_1\.venv\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
c:\gamemode_1\.venv\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [2]:
text = "This is a test document."
# ADD HERE YOUR CODE
# Perform vector search
query_vector = embedding_model.embed_query(text)

print(f"Embedding vector length: {len(query_vector)}")
print(query_vector[:10])

Embedding vector length: 768
[-0.048951826989650726, -0.039862070232629776, -0.021562794223427773, 0.009908574633300304, -0.03810390084981918, 0.012684382498264313, 0.043494462966918945, 0.07183389365673065, 0.00974861066788435, -0.006987075321376324]


#### Task 2: Generate embedding vectors with custom dataset

**Objective:**
Load custom dataset, preprocess it and generate vector embeddings.

**Task Description:**

- load pdf document "AI_Book.pdf" via langchain document loader: ``PyPDFLoader``
- use RecursiveCharacterTextSplitter to split documents into chunks
- generate embeddings for single documents

**RecursiveCharacterTextSplitter:**

This text splitter is the recommended one for generic text. It is parameterized by a list of characters. It tries to split on them in order until the chunks are small enough. The default list is `["\n\n", "\n", " ", ""]`. This has the effect of trying to keep all paragraphs (and then sentences, and then words) together as long as possible, as those would generically seem to be the strongest semantically related pieces of text.

**Useful links:**

- [Langchain PyPDFLoader](https://api.python.langchain.com/en/latest/document_loaders/langchain_community.document_loaders.pdf.PyPDFLoader.html)
- [Langchain RecursiveCharacterTextSplitter](https://api.python.langchain.com/en/latest/character/langchain_text_splitters.character.RecursiveCharacterTextSplitter.html)


In [3]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
import re

pdf_doc = "./AI_Book.pdf"

# Create pdf data loader
# ADD HERE YOUR CODE
loader = PyPDFLoader(file_path=pdf_doc)

# Load and split documents in chunks
# ADD HERE YOUR CODE
pages = loader.load()
splitter = RecursiveCharacterTextSplitter(chunk_size = 10000, chunk_overlap = 20)
pages_chunked = splitter.split_documents(pages)

# Function to clean text by removing invalid unicode characters, including surrogate pairs
def clean_text(text):
    # Remove surrogate pairs
    text = re.sub(r'[\ud800-\udfff]', '', text)
    # Optionally remove non-ASCII characters (depends on your use case)
    text = re.sub(r'[^\x00-\x7F]+', '', text)
    return text

def clean_and_create_document(chunk):
    cleaned_text = clean_text(chunk.page_content)
    return Document(page_content=cleaned_text, metadata=chunk.metadata)


pages_chunked_cleaned = [clean_and_create_document(chunk) for chunk in pages_chunked]


print(pages_chunked[0])

page_content='Aurélien GéronHands-on Machine Learning with
Scikit-Learn, Keras, and
TensorFlow
Concepts, Tools, and Techniques to
Build Intelligent SystemsSECOND EDITION
Boston Farnham Sebastopol Tokyo Beijing Boston Farnham Sebastopol Tokyo Beijing' metadata={'source': './AI_Book.pdf', 'page': 2}


In [4]:
print(pages_chunked[1])

page_content='978-1-492-03264-9
[LSI]Hands-on Machine Learning with Scikit-Learn, Keras, and TensorFlow
by Aurélien Géron
Copyright © 2019 Aurélien Géron. All rights reserved.
Printed in the United States of America.
Published by O’Reilly Media, Inc., 1005 Gravenstein Highway North, Sebastopol, CA 95472.
O’Reilly books may be purchased for educational, business, or sales promotional use. Online editions are
also available for most titles ( http://oreilly.com ). For more information, contact our corporate/institutional
sales department: 800-998-9938 or corporate@oreilly.com .
Editor:  Nicole Tache
Interior Designer:  David FutatoCover Designer:  Karen Montgomery
Illustrator:  Rebecca Demarest
June 2019:  Second Edition
Revision History for the Early Release
2018-11-05: First Release
2019-01-24: Second Release
2019-03-07: Third Release
2019-03-29: Fourth Release
2019-04-22: Fifth Release
See http://oreilly.com/catalog/errata.csp?isbn=9781492032649  for release details.
The O’Reilly logo 

In [5]:
print(f"Number of text chunks: {len(pages_chunked)}")

Number of text chunks: 501


#### Task 3: Store vector embeddings from pdf document to ChromaDB vector database.

**Objective:**

Store vector embeddings into ChromaDB to store knowledge.

**Task Description:**

- create chromadb client
- create chromadb collection
- create langchain chroma db client
- store text document chunks and vector embeddings to vector databases

**Useful links:**

- [Langchain How To](https://python.langchain.com/v0.2/docs/integrations/vectorstores/chroma/#initialization-from-client)

In [6]:
from langchain_chroma import Chroma
import chromadb
from chromadb.config import DEFAULT_TENANT, DEFAULT_DATABASE, Settings

client = chromadb.HttpClient(
    host="localhost",
    port=8000,
    ssl=False,
    headers=None,
    settings=Settings(allow_reset=True, anonymized_telemetry=False),
    tenant=DEFAULT_TENANT,
    database=DEFAULT_DATABASE,
)

# ADD HERE YOUR CODE
# Create a collection
collection = client.get_or_create_collection("collection_name")

# ADD HERE YOUR CODE
# Create chromadb
vector_db_from_client = Chroma(
    client=client,
    collection_name="collection_name",
    embedding_function=embedding_model,
    )

In [7]:
from uuid import uuid4

uuids = [str(uuid4()) for _ in range(len(pages_chunked_cleaned[:50]))]

vector_db_from_client.add_documents(documents=pages_chunked_cleaned[:50], id = uuids)


#uuids = [str(uuid4()) for _ in range(len(pages_chunked_cleaned[:50]))]

# ADD HERE YOUR CODE
#vector_db_from_client.add_texts(pages_chunked_cleaned, id = uuids)

['c6faad1f-cda5-4a39-b04a-d64a223e58f5',
 'fb520e67-2bd3-4d19-81f0-db2b3f3d2cd0',
 'bdfce2da-329d-48d5-8bf6-1dbace21f8c5',
 'e8cc0431-dbca-45bf-a8c1-df2fe4b91e70',
 '80eeb072-c29a-4e1d-a6ba-1999febe62f1',
 '1ba8ae1b-9b03-4694-8135-90f37f3ae0ac',
 '92e41fd1-784b-4646-815a-5e46da83faa8',
 '16232786-a5b8-4b35-b297-8b765b2bda63',
 'ce6a86d4-0f47-4808-adee-2ee2f63e5092',
 '756c06e5-261e-4b8f-b1d7-6305c46a567a',
 '2f5aba46-ea85-418f-a735-c8fd3a61af36',
 'de116e5a-0c9f-44af-b2e2-ea2188310a1b',
 'cf17538f-5d64-48f8-8005-b7c639bf6e2c',
 'bdf1abf8-b70f-4f7d-b7ad-a1e318dee66d',
 '991dd677-22ee-4b0b-995c-0cfa8f0c35b6',
 'd8ce1875-21e5-4bdd-b4f2-ec1af1a2e4a1',
 'f25dc4a0-aabc-4110-bb95-de9615db2b10',
 '90f6f793-b6de-482a-805d-c62da9bda6fc',
 'd5a9c88f-73d5-4210-916f-09709c996981',
 '4730761e-e38b-44de-bc34-adca9318660d',
 'ee72100e-3df1-4280-9172-f914c17292a8',
 'ea7d5585-e644-4057-a26f-72bc8195bfe1',
 '0919b2ea-4d92-468a-97a3-fe8c0c3af0bc',
 '4b17753f-e99c-4d58-bb12-ee075d0c3b9e',
 'cff6a5e8-d337-

In [8]:
client.count_collections()

1

In [9]:
# client.delete_collection("ai_model_book")

#### Task 4: Access ChromaDB and perform vector search

**Objective:**

Use query to perform vector search against ChromaDB vector database

**Task Description:**

- define query
- run vector search
- print k=3 most similar documents


**Useful links:**

- [Langchain Query ChromaDB](https://python.langchain.com/v0.2/docs/integrations/vectorstores/chroma/#query-directly)

In [10]:
search_query = str("Types of Machine Learning Systems")

results = vector_db_from_client.similarity_search(
    "search_query",
    k=3,
    )

for res in results:
    print(res.page_content)
    print(res.metadata)
    print("\n")

Frame the Problem                                                                                                    39
Select a Performance Measure                                                                                  42
Check the Assumptions                                                                                             45
Get the Data                                                                                                                    45
Create the Workspace                                                                                                45
Download the Data                                                                                                    49
Take a Quick Look at the Data Structure                                                                50
Create a Test Set                                                                                                          54
Discover and Visualize the Data to Gain Insights